一个简单的示例，其中两个 agent 从 10 倒数到 1。

我们首先定义 agent 类及其各自的处理消息的程序。 

我们创建两个 agent 类：Modifier 和 Checker。

Modifier agent 修改给定的数字，而 Check agent 根据条件检查该值。 

我们还创建了一个 Message 数据类，它定义了在 agent 之间传递的消息。

In [1]:
from dataclasses import dataclass
from typing import Callable

from autogen_core import DefaultTopicId, MessageContext, RoutedAgent, default_subscription, message_handler


@dataclass
class Message:
    content: int


@default_subscription
class Modifier(RoutedAgent):
    def __init__(self, modify_val: Callable[[int], int]) -> None:
        super().__init__("A modifier agent.")
        self._modify_val = modify_val

    @message_handler
    async def handle_message(self, message: Message, ctx: MessageContext) -> None:
        val = self._modify_val(message.content)
        print(f"{'-'*80}\nModifier:\nModified {message.content} to {val}")
        await self.publish_message(Message(content=val), DefaultTopicId())  # type: ignore


@default_subscription
class Checker(RoutedAgent):
    def __init__(self, run_until: Callable[[int], bool]) -> None:
        super().__init__("A checker agent.")
        self._run_until = run_until

    @message_handler
    async def handle_message(self, message: Message, ctx: MessageContext) -> None:
        if not self._run_until(message.content):
            print(f"{'-'*80}\nChecker:\n{message.content} passed the check, continue.")
            await self.publish_message(Message(content=message.content), DefaultTopicId())
        else:
            print(f"{'-'*80}\nChecker:\n{message.content} failed the check, stopping.")


您可能已经注意到，agent 的逻辑（无论是使用模型还是代码执行器）与消息的传递方式完全分离。 

这是核心思想：框架提供通信基础设施，agent 负责自己的逻辑。 我们将通信基础设施称为 **Agent 运行时**。

Agent 运行时是此框架的关键概念。 除了传递消息之外，它还管理 agent 的生命周期。 因此，agent 的创建由运行时处理。

In [2]:
from autogen_core import AgentId, SingleThreadedAgentRuntime

# Create a local embedded runtime.
runtime = SingleThreadedAgentRuntime()

# Register the modifier and checker agents by providing
# their agent types, the factory functions for creating instance and subscriptions.
await Modifier.register(
    runtime,
    "modifier",
    # Modify the value by subtracting 1
    lambda: Modifier(modify_val=lambda x: x - 1),
)

await Checker.register(
    runtime,
    "checker",
    # Run until the value is less than or equal to 1
    lambda: Checker(run_until=lambda x: x <= 1),
)

# Start the runtime and send a direct message to the checker.
runtime.start()
await runtime.send_message(Message(10), AgentId("checker", "default"))
await runtime.stop_when_idle()


--------------------------------------------------------------------------------
Checker:
10 passed the check, continue.
--------------------------------------------------------------------------------
Modifier:
Modified 10 to 9
--------------------------------------------------------------------------------
Checker:
9 passed the check, continue.
--------------------------------------------------------------------------------
Modifier:
Modified 9 to 8
--------------------------------------------------------------------------------
Checker:
8 passed the check, continue.
--------------------------------------------------------------------------------
Modifier:
Modified 8 to 7
--------------------------------------------------------------------------------
Checker:
7 passed the check, continue.
--------------------------------------------------------------------------------
Modifier:
Modified 7 to 6
--------------------------------------------------------------------------------
Checker:


从 agent 的输出中，我们可以看到该值已成功从 10 递减到 1，正如 modifier 和 checker 条件所规定。

AutoGen 还支持分布式 agent 运行时，它可以托管在不同进程或机器上运行的具有不同身份、语言和依赖项的 agent。

https://msdocs.cn/autogen/stable/user-guide/core-user-guide/design-patterns/intro.html

autogen 多代理模式的几个典型应用场景： 工作流、群聊、辩论、反思